# **Тренировка модели для задачи получения предсказания**

##**1. Импортируем библиотеки**

In [1]:
import warnings
warnings.simplefilter("ignore")

In [6]:
!pip install --upgrade pip
!pip install torch==1.13.1 \
    torchdata==0.5.1 --quiet
!pip install \
    transformers==4.33.1 \
    datasets==2.11.0 \
    evaluate==0.4.0 \
    rouge_score==0.1.2 \
    loralib==0.1.1 \
    peft==0.3.0 --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 9.2 MB/s eta 0:00:00:00:010:01
  Attempting uninstall: pip
    Found existing installation: pip 23.3.2
    Uninstalling pip-23.3.2:
      Successfully uninstalled pip-23.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
apache-beam 2.46.0 requires dill<0.3.2,>=0.3.1.1, but you have dill 0.3.6 which is incompatible.
apache-beam 2.46.0 requires pyarrow<10.0.0,>=3.0.0, but you have pyarrow 11.0.0 which is incompatible.
pathos 0.3.1 requires dill>=0.3.7, but you have dill 0.3.6 which is incompatible.
pathos 0.3.1 requires multiprocess>=0.70.15, but you have multiprocess 0.70.14 which is incompatible.


In [9]:
from datasets import load_dataset
from transformers import AutoModelForCausalLM, AutoTokenizer, GenerationConfig, Trainer, TrainingArguments
import torch
import time
#import evaluate
import pandas as pd
import numpy as np

##**2. Загружаем датасет для fine-tuning тренировки**

In [ ]:
%%bash 
export HF_DATASETS_CACHE="/kaggle/working/"

In [4]:
pip install -U datasets

  Using cached datasets-2.19.0-py3-none-any.whl.metadata (19 kB)
Using cached datasets-2.19.0-py3-none-any.whl (542 kB)
  Attempting uninstall: datasets
    Found existing installation: datasets 2.11.0
    Uninstalling datasets-2.11.0:
      Successfully uninstalled datasets-2.11.0
Note: you may need to restart the kernel to use updated packages.


In [3]:
import pandas as pd
dataset = pd.read_csv('/kaggle/input/daily-horoscopes-collection/fin_horo.csv')
dataset = dataset.drop(['date', 'len', 'source'], axis=1)
dataset.head()
#делаю стратифицированную выборку
df = dataset.groupby('zodiac', group_keys= False ).apply ( lambda x: x. sample (500))
df.shape
#3000 train and 3000 test  
#нужно перевести в формат dataset.dict  и добавить также тестовую выборку
#df_train = df.sample(frac=0.5)
#df_test = df.sample(frac=0.5) #не факт, что выборки не пересекаются 

/tmp/ipykernel_34/3916947399.py:6: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df = dataset.groupby('zodiac', group_keys= False ).apply ( lambda x: x. sample (500))


(6000, 2)

In [4]:
#!import test train cross validation 
from sklearn.model_selection import train_test_split

df_train, df_test = train_test_split(dataset, random_state=104, test_size=0.25, shuffle=True)

In [5]:
import datasets
from datasets import Dataset

dataset = datasets.DatasetDict(
    {
        "train": Dataset.from_pandas(df_train),
        "test": Dataset.from_pandas(df_test),
    }
)

dataset 

DatasetDict({
    train: Dataset({
        features: ['zodiac', 'horo', '__index_level_0__'],
        num_rows: 181138
    })
    test: Dataset({
        features: ['zodiac', 'horo', '__index_level_0__'],
        num_rows: 60380
    })
})

In [ ]:
"""
from datasets import load_dataset
huggingface_dataset_name = "dkagramanyan/horoscopes_ru" 
dataset = load_dataset(huggingface_dataset_name)
dataset
"""

**Подготовка датасета для входа модели**

In [10]:
#map делает операции сразу и над train, и над test
#загружаем токенайзер
tokenizer = AutoTokenizer.from_pretrained('AlexWortega/wortegaLM',padding_side='left')
device = 'cuda'

tokenizer_config.json:   0%|          | 0.00/347 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/120 [00:00<?, ?B/s]

In [11]:
def fix_tokenizer(tokenizer):
    # Fixing broken tokenisers
    special_tokens = dict()
    for token_id in range(1000):
        token = tokenizer.convert_ids_to_tokens(token_id)
        if tokenizer.pad_token_id in (None, tokenizer.vocab_size) and "pad" in token:
            special_tokens["pad_token"] = token
        if tokenizer.bos_token_id in (None, tokenizer.vocab_size) and "<s>" in token:
            special_tokens["bos_token"] = token
        if tokenizer.eos_token_id in (None, tokenizer.vocab_size) and "</s>" in token:
            special_tokens["eos_token"] = token
        if tokenizer.unk_token_id in (None, tokenizer.vocab_size) and "unk" in token:
            special_tokens["unk_token"] = token
        if tokenizer.sep_token_id in (None, tokenizer.vocab_size) and "sep" in token:
            special_tokens["sep_token"] = token

    if tokenizer.sep_token_id in (None, tokenizer.vocab_size) and "bos_token" in special_tokens:
        special_tokens["sep_token"] = special_tokens["bos_token"]

    if tokenizer.pad_token_id in (None, tokenizer.vocab_size) and "pad_token" not in special_tokens:
        if tokenizer.unk_token_id is not None:
            special_tokens["pad_token"] = tokenizer.unk_token
        else:
            special_tokens["pad_token"] = "<|pad|>"

    if tokenizer.sep_token_id in (None, tokenizer.vocab_size) and "sep_token" not in special_tokens:
        if tokenizer.bos_token_id is not None:
            special_tokens["sep_token"] = tokeniser.bos_token
        else:
            special_tokens["sep_token"] = "<|sep|>"

    tokenizer.add_special_tokens(special_tokens)

    return tokenizer

In [12]:
def preprocessing(examples): #examples - это словарь с date, sign, text
    #examples['sign'] = examples['sign'].replace("scorpio", "скорпион")
    examples['Prediction'] = f'Предсказание судьбы для знака зодиака {examples["zodiac"]} {examples["horo"]}' #prompt
    tokens = tokenizer(examples['Prediction'], truncation=False, return_tensors=None)
    tokens['labels'] = tokens['input_ids'].copy()

    return tokens


In [13]:
dataset= dataset.map(
       lambda examples: preprocessing(examples),
       batched=False,
       remove_columns = dataset['train'].column_names
)


  0%|          | 0/181138 [00:00<?, ?ex/s]

  0%|          | 0/60380 [00:00<?, ?ex/s]

In [ ]:
#dataset['train'][0]

##**3. Загружаем модель**

In [15]:
#загружаем модель
model_name = AutoModelForCausalLM.from_pretrained('AlexWortega/wortegaLM')

model_name.resize_token_embeddings(len(tokenizer))
model_name.to('cuda')

#device = 'cuda'
#model_name.to(device)

#import torch

#device = 'cuda' if torch.cuda.is_available() else 'cpu'
#model.to(device)
#data = data.to(device)


You are resizing the embedding layer without providing a `pad_to_multiple_of` parameter. This means that the new embedding dimension will be 30000. This might induce some performance reduction as *Tensor Cores* will not be available. For more details about this, or help on choosing the correct value for resizing, refer to this guide: https://docs.nvidia.com/deeplearning/performance/dl-performance-matrix-multiplication/index.html#requirements-tc


GPTNeoForCausalLM(
  (transformer): GPTNeoModel(
    (wte): Embedding(30000, 768)
    (wpe): Embedding(2048, 768)
    (drop): Dropout(p=0.0, inplace=False)
    (h): ModuleList(
      (0): GPTNeoBlock(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPTNeoAttention(
          (attention): GPTNeoSelfAttention(
            (attn_dropout): Dropout(p=0.0, inplace=False)
            (resid_dropout): Dropout(p=0.0, inplace=False)
            (k_proj): Linear(in_features=768, out_features=768, bias=False)
            (v_proj): Linear(in_features=768, out_features=768, bias=False)
            (q_proj): Linear(in_features=768, out_features=768, bias=False)
            (out_proj): Linear(in_features=768, out_features=768, bias=True)
          )
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPTNeoMLP(
          (c_fc): Linear(in_features=768, out_features=3072, bias=True)
          (c_proj): Linear(in_features=3

In [ ]:
generate_seqs('Что бы мне поделать', model_name, k=2) #проверили, что моделька работает

##**5. Fine-tuning модели при помощи PEFT/LoRA**

In [16]:
!pip install peft

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [17]:
from peft import LoraConfig, get_peft_model, TaskType

lora_config = LoraConfig(
    r=8,  #rank
    lora_alpha=32, #количество матриц уточнить
    target_modules=['q_proj', 'v_proj'], #формула селф аттеншн, изучить  (еще есть k, o) можно напечатать модель и посмотреть как ее слои называются
    lora_dropout=0.05, 
    bias='none',
    task_type = TaskType.CAUSAL_LM #FLAN-T5 TODO: заменить CasualLLM (в Peft repo)
)

In [19]:
#добавляем параметры к оригинальной LLM
peft_model = get_peft_model(model_name,
                            lora_config) #заменяет self-attention слой на лора адаптер

In [20]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(
            tokenizer=tokenizer,
            model=model_name,
            padding=True,
            return_tensors="pt"
        )

In [21]:
output_dir = f'./peft-model'

peft_training_args = TrainingArguments( #поэспериментировать
    output_dir = output_dir,
    auto_find_batch_size = True,
    learning_rate=3e-5,
    num_train_epochs=5, #поэспериментировать
    logging_steps=1
)

peft_trainer = Trainer(
    data_collator = data_collator, #learning rate, batch size
    model=peft_model,
    args=peft_training_args,
    eval_dataset = dataset['test'],
    train_dataset = dataset['train']
)



In [ ]:

#import torch

#device = 'cuda' if torch.cuda.is_available() else 'cpu'
#model_name.to(device)
#data = data.to(device)


In [25]:
!pip install wandb --upgrade

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 10.1 MB/s eta 0:00:0000:010:01
  Attempting uninstall: wandb
    Found existing installation: wandb 0.16.2
    Uninstalling wandb-0.16.2:
      Successfully uninstalled wandb-0.16.2


In [27]:
#тренируем и сохраняем результат
peft_trainer.train()

peft_model_path = "./peft-model-checkpoint-local"
peft_trainer.model.save_pretrained(peft_model_path)
tokenizer.save_pretrained(peft_model_path)

#7357873d33ffb30798ceba7e13a639b0560dfe05

RuntimeError: Caught RuntimeError in replica 1 on device 1.
Original Traceback (most recent call last):
  File "/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/parallel_apply.py", line 64, in _worker
    output = module(*input, **kwargs)
  File "/opt/conda/lib/python3.10/site-packages/torch/nn/modules/module.py", line 1194, in _call_impl
    return forward_call(*input, **kwargs)
  File "/opt/conda/lib/python3.10/site-packages/peft/peft_model.py", line 678, in forward
    return self.base_model(
  File "/opt/conda/lib/python3.10/site-packages/torch/nn/modules/module.py", line 1194, in _call_impl
    return forward_call(*input, **kwargs)
  File "/opt/conda/lib/python3.10/site-packages/transformers/models/gpt_neo/modeling_gpt_neo.py", line 748, in forward
    transformer_outputs = self.transformer(
  File "/opt/conda/lib/python3.10/site-packages/torch/nn/modules/module.py", line 1194, in _call_impl
    return forward_call(*input, **kwargs)
  File "/opt/conda/lib/python3.10/site-

In [22]:
config = dict(
    tokeniser_kwargs={ #меняем параметры
        'max_length': 2048,
        'truncation': False,
        'padding': True,
        'return_tensors': "pt",
    },
    generation_kwargs={
        "max_new_tokens": 512,
        "eos_token_id": tokenizer.eos_token_id,
        "repetition_penalty": 1.2,
        "top_k": 50,
        "temperature": 1.1,
        "tOp_p": 0.92,
        "num_beams": 2,
        "use_cache": True,
        "do_sample": False,
    }
)

In [23]:
def generate(text, model, tokenizer, config):
    tokens = tokenizer(text, **config["tokeniser_kwargs"])
    with torch.inference_mode():
        generate_result = (
            model.generate(
                input_ids=tokens["input_ids"].to(model.device),
                attention_mask=tokens["attention_mask"].to(model.device),
                **config["generation_kwargs"]
            )
            .cpu()
            .numpy()
        )

    decoded_tokens = [
        tokenizer.decode(r, skip_special_tokens=True) for r in generate_result
    ]
    
    return decoded_tokens

In [ ]:
#Генерируем предсказание с нужным текстом

generate('🔮 Гадалка Агата говорит, что', peft_model, k=1)

##**6. Готовим модель для инференса**

In [ ]:
from peft import PeftModel, PeftConfig

peft_model_base = AutoModelForCausalLM.from_pretrain('AlexWortega/wortegaLM', torch_dtype=torch.bfloat16) #torch_dtype убрать?
tokenizer = AutoTokenizer.from_pretrained('AlexWortega/wortegaLM')

peft_model = PeftModel.from_pretrained(peft_model_base,
                                       "./peft-model-checkpoint-local",
                                       torch_dtype=torch.bfloat16,
                                       is_trainable = False)

**7. Проверяем качество **